In [2]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [1]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = 'xxx'

# the API call
query = 'factory+automation' # topic is "large language model"factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [2]:
import openai

openai.api_key = 'xxx'

#System prompt (for all users)
system_prompt = f"""There is a list of Given Solutions with short description: Dynamic Process Monitoring System	Implement a real-time monitoring system using sensors and IoT devices to collect data from various stages of the production process. This system analyzes the data to provide actionable insights, optimizing production efficiency, identifying bottlenecks, and preventing delays.
Self-Optimizing Production Lines	Develop production lines equipped with AI and machine learning algorithms that continuously analyze and optimize production parameters in real-time. These lines automatically adjust settings to maximize efficiency and reduce delays.
Intelligent Production Scheduling	Implement an AI-driven scheduling system that considers real-time data like machine availability and order priorities. This system optimizes the production schedule to minimize idle time and ensure timely delivery.
Smart Inventory Management	Utilize AI and predictive analytics to optimize inventory levels based on demand forecasts and production schedules. This system automates replenishment orders, reducing stockouts and improving efficiency.
Collaborative Robotics in Production	Introduce collaborative robots (cobots) to work alongside humans, performing repetitive tasks efficiently. These cobots are equipped with sensors and AI to ensure safe and efficient collaboration.
Intelligent Production Control Algorithm	Create an AI and machine learning-based algorithm to dynamically adjust production plans and resource allocation, minimizing downtime and maximizing throughput.
Predictive Maintenance using AI	Develop a system that uses AI to monitor machine health in real-time, predicting maintenance needs to minimize downtime and optimize schedules.
Digital Twin Simulation for Production Optimization	Create a virtual simulation platform to test process optimizations and identify inefficiencies before making actual production line changes, enhancing productivity and profitability.
Automated Quality Control System	Create a system integrating computer vision and AI to perform real-time inspection of production processes and products, detecting defects and enabling immediate corrective actions.
Intelligent Inventory Management System	Implement a system using AI and predictive algorithms to optimize inventory levels, enhance supply chain efficiency, and minimize inventory costs.
        """

#Iteration1 prompt (Improve First Time / CTO)
iteration1_solutions = {}
i = 0

for doc in mergeflow_docs: #stores all the problems in 1 string 
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```    
  
    Imagine you are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Focus on Sense&Act.
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Core business: Sense&Act, PMA, Scada, PLCs.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Find products not already on the market, they have to be new and differenciators
        - Ensure all initiatives align with business profitability.
        - Make investment decisions as if using personal funds.
        - Aim for non-cannibalizing, fringe markets.
        - PLCs are scalable; seek similar scalability in AI applications.
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        - Your work is highly cruical and you are responsible of coming up with the next best innovation.

        Now take a break, consider your rolle and improve all the given possible solutions according to the given values the CTO follows.
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
    print("\n---\n")  # This adds a separator between the messages for readability
    iteration1_solutions[i] = completion.choices[0].message
    i += 1


#Iteration2 prompt (Improve Second Time / Pain Points Clients)
iteration2_solutions = {}

user_prompt1 = f"""

 Imagine yourself as the central figure in Siemens' client relations landscape, tasked with the pivotal role of overseeing and resolving all customer-related issues. You have an in-depth understanding of the various challenges and obstacles your clients face, possessing a comprehensive grasp of their pain points, which range from technical difficulties to service delivery inefficiencies. With this profound insight, you are now presented with an array of innovative solutions proposed by the Chief Technology Officer (CTO).

Your mission is to meticulously evaluate these new solutions through the lens of the unique needs and expectations of Siemens' diverse clientele. Consider how each proposed innovation aligns with the specific pain points you've identified, and critically assess their potential to not only address but also enhance the overall client experience. Your goal is to refine and adapt these solutions, ensuring they are perfectly tailored to meet, if not exceed, Siemens' clients' expectations.

In doing so, focus on optimizing the integration of these solutions into the existing Siemens ecosystem, ensuring they are both practical and scalable. Your analysis should consider the feasibility, cost-effectiveness, and potential impact on client satisfaction and loyalty. You are encouraged to think creatively, leveraging your comprehensive understanding of client issues to propose modifications or entirely new solutions that could better serve Siemens' clientele.

This task requires a blend of technical acumen, strategic thinking, and a deep empathy for the client experience. As you undertake this challenge, remember that your ultimate objective is to elevate Siemens' reputation for customer-centric innovation, securing its position as a leader in delivering exceptional client service and support.
    
    """
    

completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0613",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
iteration2_solutions[0] = completion.choices[0].message


#Iteration3 prompt (Improve Third Time / CFO)
iteration3_solutions = {}

user_prompt2 = f"""

    Transitioning from the role of a Chief Technology Officer (CTO) to that of a Chief Financial Officer (CFO) shifts the focus from technological innovation and product development to financial strategy, risk management, and the economic feasibility of new business ideas. As a CFO with a perspective centered on financial health and profitability, your guiding principles when evaluating a new business idea would include:

    - Financial Viability and Risk Assessment: Evaluate the cost of development, operational expenses, and the risk profile of the idea. Ensure the financial models account for worst-case scenarios, and the potential returns justify the risks.
    - Capital Allocation and Investment Efficiency: Prioritize projects with the best risk-adjusted returns. Investment should not only be limited by a fixed amount but also assessed for efficiency, ensuring the capital is allocated to opportunities with the highest potential for profitability and growth.
    - Revenue and Profit Margins: Target projects that promise to meet or exceed a certain threshold in annual revenue with sustainable profit margins. Scrutinize the sales forecasts and market penetration strategies to ensure they are realistic and attainable.
    - Cost Management: Emphasize the importance of keeping development and operational costs under control. Explore strategies for quick, low-investment testing of business ideas to minimize financial exposure.
    - Financial Commitments: Assess the financial commitments required for each project, including initial investment, ongoing operational costs, and any potential future expenses related to scaling the business.
    - Return on Investment (ROI): Calculate the expected ROI for each initiative, considering both direct financial returns and indirect benefits (e.g., brand strengthening, market position improvement).
    - Cash Flow Management: Ensure that the project timelines and milestones align with cash flow projections to maintain financial stability.
    - Market Differentiation and Competition: Evaluate the competitive landscape and the potential for the new product to differentiate itself in the market. Consider the financial impact of entering competitive markets versus creating new niches.
    - Strategic Fit and Long-term Growth: Assess how the new idea fits within the broader financial goals and strategies of the company. Consider its contribution to long-term growth and sustainability.
    - Stakeholder Alignment: Align the project with the expectations and requirements of key stakeholders, including shareholders, board members, and financial institutions.
    - Regulatory and Compliance Costs: Consider the financial implications of regulatory compliance and the costs associated with maintaining those standards.
    - Customer Validation and Market Demand: Validate the market demand through financial lenses, such as willingness to pay, customer acquisition cost, and lifetime value.
    
Improving the idea with a CFO perspective might involve refining the business model to enhance revenue streams, reducing costs through more efficient operations or strategic partnerships, and ensuring the financial metrics align with corporate objectives. Additionally, exploring financing options that balance equity and debt to fund the project while minimizing financial strain on the company would be crucial."""
    

completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0613",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
iteration3_solutions[0] = completion.choices[0].message


#Summary

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about the three different improvements to have a final list of solutions with short descriptions. 
    
       
    Please summarize in a table given:
    1) Name of Idea
    2) Short Descriptions (30 words)
    3) Value add for user
    4) How it works
   
    """
    

completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0613",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

Given the guiding principles followed by the CTO, let's review and improve the given possible solutions in the context of factory automation:

1. Dynamic Process Monitoring System:
   - Enhance the system to focus on real-time monitoring of critical process parameters that directly impact production efficiency and quality.
   - Prioritize quick implementation and low investment by leveraging existing sensors and IoT devices.
   - Aim for large profits by optimizing production efficiency and preventing delays.
   - Validate the effectiveness of the system through pilot testing with business units and customer feedback.

2. Self-Optimizing Production Lines:
   - Develop production lines with AI and machine learning algorithms that optimize production parameters based on real-time data.
   - Prioritize scalability by focusing on LLMs in Engineering and Operations to ensure compatibility and easy integration with existing systems.
   - Aim for large profits by maximizing efficiency and red